In [22]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import chi2_contingency
from sklearn.manifold import MDS

## Added on 2024-11-14
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn.model_selection import train_test_split

## Added on 2024-11-15
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
'''
# Definition of paths
data_dir = os.path.join(os.getcwd(), '../Backup_Dataset')
print(data_dir)

raw_data_dir = os.path.join(data_dir, 'raw')
ids_mapping = os.path.join(raw_data_dir, 'IDS_mapping.csv')
diabetes_data = os.path.join(raw_data_dir, 'diabetic_data.csv')

mapping_data_dir = os.path.join(data_dir, 'Mapping')
os.makedirs(mapping_data_dir, exist_ok=True)

preprocessed_out_dir = os.path.join(data_dir, 'Preprocessed')
os.makedirs(preprocessed_out_dir, exist_ok=True)

admission_type_mapping_data = os.path.join(mapping_data_dir, 'admission_type_mapping.csv')
discharge_disposition_data = os.path.join(mapping_data_dir, 'discharge_disposition_mapping.csv')
admission_source_mapping_data = os.path.join(mapping_data_dir, 'admission_source_mapping.csv')

preprocessed_saved_filepath = os.path.join(preprocessed_out_dir, 'preprocessed.pkl')
'''

"\n# Definition of paths\ndata_dir = os.path.join(os.getcwd(), '../Backup_Dataset')\nprint(data_dir)\n\nraw_data_dir = os.path.join(data_dir, 'raw')\nids_mapping = os.path.join(raw_data_dir, 'IDS_mapping.csv')\ndiabetes_data = os.path.join(raw_data_dir, 'diabetic_data.csv')\n\nmapping_data_dir = os.path.join(data_dir, 'Mapping')\nos.makedirs(mapping_data_dir, exist_ok=True)\n\npreprocessed_out_dir = os.path.join(data_dir, 'Preprocessed')\nos.makedirs(preprocessed_out_dir, exist_ok=True)\n\nadmission_type_mapping_data = os.path.join(mapping_data_dir, 'admission_type_mapping.csv')\ndischarge_disposition_data = os.path.join(mapping_data_dir, 'discharge_disposition_mapping.csv')\nadmission_source_mapping_data = os.path.join(mapping_data_dir, 'admission_source_mapping.csv')\n\npreprocessed_saved_filepath = os.path.join(preprocessed_out_dir, 'preprocessed.pkl')\n"

In [13]:
# Define the base directory using the current working directory
current_dir = os.getcwd()
backup_dir = os.path.join(current_dir, 'Backup_Dataset', 'Preprocessed')

# File paths for both PKL files
preprocessed_filepath = os.path.join(backup_dir, 'preprocessed.pkl')
transformed_filepath = os.path.join(backup_dir, 'transformed_data.pkl')

# Check if both files exist and load them if they do
if not os.path.exists(preprocessed_filepath):
    print(f"File does not exist: {preprocessed_filepath}")
else:
    df_preprocessed = pd.read_pickle(preprocessed_filepath)
    print("Loaded 'preprocessed.pkl' successfully.")

# Save 'transformed_data.pkl' in the same directory
# Assuming df_transformed is the DataFrame you want to save
# Uncomment the following lines if you have df_transformed ready to save
# df_transformed.to_pickle(transformed_filepath)
# print(f"'transformed_data.pkl' saved successfully at {transformed_filepath}")


File does not exist: C:\Users\amitniti\OneDrive - Intel Corporation\Documents\AJ-Personal\(INTEL) IISc-MTech\2-Data Science in Practice\Project-1-DSiP-DiabetesReadmit\ANJ_GITclone\2024-11-20\Backup_Dataset\Preprocessed\preprocessed.pkl


In [5]:
# Directly specify the full path including the date directory
data_dir = r"C:\Users\amitniti\OneDrive - Intel Corporation\Documents\AJ-Personal\(INTEL) IISc-MTech\2-Data Science in Practice\Project-1-DSiP-DiabetesReadmit\ANJ_GITclone\2024-11-14\Backup_Dataset"
preprocessed_saved_filepath = os.path.join(data_dir, 'Preprocessed', 'preprocessed.pkl')

# Check if the path exists
if not os.path.exists(preprocessed_saved_filepath):
    print(f"File does not exist: {preprocessed_saved_filepath}")
else:
    # Load the pickle file
    df = pd.read_pickle(preprocessed_saved_filepath)
    print("Dataframe loaded successfully.")

Dataframe loaded successfully.


In [8]:
print("Shape: ",df.shape) #(69987, 44)
print()

df.info()

Shape:  (69987, 44)

<class 'pandas.core.frame.DataFrame'>
Index: 69987 entries, 0 to 101765
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   encounter_id              69987 non-null  int64   
 1   patient_nbr               69987 non-null  int64   
 2   race                      69987 non-null  category
 3   gender                    69987 non-null  object  
 4   age                       69987 non-null  object  
 5   admission_type_id         69987 non-null  category
 6   discharge_disposition_id  69987 non-null  category
 7   admission_source_id       69987 non-null  category
 8   time_in_hospital          69987 non-null  int64   
 9   medical_specialty         69987 non-null  category
 10  num_lab_procedures        69987 non-null  int64   
 11  num_procedures            69987 non-null  int64   
 12  num_medications           69987 non-null  int64   
 13  number_outpatient         699

### Numerical Column Analysis ###

In [9]:
numerical_cols = [
  'num_lab_procedures', 'num_medications', 'num_procedures', 'number_diagnoses', 'number_inpatient', 'number_outpatient', 'time_in_hospital'
]

In [10]:
df[numerical_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
num_lab_procedures,69987.0,42.875863,19.894291,1.0,31.0,44.0,57.0,132.0
num_medications,69987.0,15.665281,8.287321,1.0,10.0,14.0,20.0,81.0
num_procedures,69987.0,1.425379,1.757003,0.0,0.0,1.0,2.0,6.0
number_diagnoses,69987.0,7.224170,2.001257,1.0,6.0,8.0,9.0,16.0
number_inpatient,69987.0,0.176276,0.601633,0.0,0.0,0.0,0.0,12.0
number_outpatient,69987.0,0.279595,1.064108,0.0,0.0,0.0,0.0,42.0
time_in_hospital,69987.0,4.273336,2.933987,1.0,2.0,3.0,6.0,14.0


In [11]:
scaler = MinMaxScaler()

# Fit and transform the numerical columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
df

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,max_glu_serum_measured,A1Cresult_measured,repaglinide_measured,glimepiride_measured,rosiglitazone_measured,pioglitazone_measured,glyburide_measured,glipizide_measured,metformin_measured,insulin_measured
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,0.000000,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,No
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,0.153846,Unknown,...,No,No,No,No,No,No,No,No,No,Yes
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,0.076923,Unknown,...,No,No,No,No,No,No,No,Yes,No,No
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,0.076923,Unknown,...,No,No,No,No,No,No,No,No,No,Yes
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,0.000000,Unknown,...,No,No,No,No,No,No,No,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101754,443842016,183087545,Caucasian,Female,[70-80),1,1,7,0.615385,Unknown,...,No,Yes,No,No,No,No,Yes,No,No,Yes
101755,443842022,188574944,Other,Female,[40-50),1,1,7,1.000000,Unknown,...,No,Yes,No,No,No,No,No,Yes,No,Yes
101756,443842070,140199494,Other,Female,[60-70),1,1,7,0.076923,Unknown,...,No,No,No,No,No,No,No,No,No,Yes
101758,443842340,120975314,Caucasian,Female,[80-90),1,1,7,0.307692,Unknown,...,No,No,No,No,No,No,No,No,No,Yes


In [12]:
current_dir = os.getcwd()
print(current_dir)

C:\Users\amitniti\OneDrive - Intel Corporation\Documents\AJ-Personal\(INTEL) IISc-MTech\2-Data Science in Practice\Project-1-DSiP-DiabetesReadmit\ANJ_GITclone\2024-11-20


#### Additional Polynomial Features (Squared and Square Root) #### 

In [23]:
# Separate features and target variable
X = df.drop(columns=['readmitted'])  # Assuming 'readmitted' is the target
y = df['readmitted']

# Encode the target variable if it is categorical
if y.dtype == 'object' or isinstance(y.iloc[0], str):
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

# Step 1: Feature Scaling and Polynomial Feature Engineering

# Identify numerical columns for scaling and polynomial features
numerical_cols = [
    'num_lab_procedures', 'num_medications', 'num_procedures',
    'number_diagnoses', 'number_inpatient', 'number_outpatient', 'time_in_hospital'
]

# Handle missing values in numerical columns
X[numerical_cols] = X[numerical_cols].fillna(X[numerical_cols].median())

# Generate Polynomial Features (Squared and Square Root)
for col in numerical_cols:
    X[f'{col}_squared'] = X[col] ** 2
    X[f'{col}_sqrt'] = np.sqrt(X[col].clip(lower=0))  # Replace negatives with 0

# Standard Scaling
scaler = StandardScaler()
features_to_scale = numerical_cols + [f'{col}_squared' for col in numerical_cols] + [f'{col}_sqrt' for col in numerical_cols]
X[features_to_scale] = scaler.fit_transform(X[features_to_scale])

# Step 2: One-Hot Encoding for Categorical Features
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Step 3: Feature Selection

# (a) Remove Highly Correlated Features
correlation_matrix = X.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_corr_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.9)]
X = X.drop(columns=high_corr_features)

# (b) Feature Importance using RandomForestClassifier
# Split data for feature importance calculation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit a Random Forest model to get feature importances
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.DataFrame(
    rf_model.feature_importances_, index=X.columns, columns=['Importance']
).sort_values(by='Importance', ascending=False)

# Select top k features based on importance (e.g., k=20)
k = 20
top_features = feature_importances.head(k).index
X = X[top_features]

# Step 4: Create In-Memory Transformed DataFrame
transformed_data_df = pd.concat([X, pd.Series(y, name='readmitted')], axis=1)

print("Feature scaling and selection with polynomial features completed. Transformed data is ready in 'transformed_data_df'.")

Feature scaling and selection with polynomial features completed. Transformed data is ready in 'transformed_data_df'.


In [24]:
print("Shape: ", transformed_data_df.shape)
print()

print(transformed_data_df.info())

Shape:  (89319, 21)

<class 'pandas.core.frame.DataFrame'>
Index: 89319 entries, 0 to 69985
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   encounter_id               69987 non-null  float64
 1   patient_nbr                69987 non-null  float64
 2   num_lab_procedures         69987 non-null  float64
 3   num_medications            69987 non-null  float64
 4   time_in_hospital           69987 non-null  float64
 5   number_diagnoses           69987 non-null  float64
 6   num_procedures             69987 non-null  float64
 7   insulin                    69987 non-null  float64
 8   gender_Male                69987 non-null  object 
 9   A1Cresult                  69987 non-null  float64
 10  diag_2_Other               69987 non-null  object 
 11  diag_3_Other               69987 non-null  object 
 12  number_inpatient_squared   69987 non-null  float64
 13  metformin                  699

In [21]:
transformed_data_df

,encounter_id,patient_nbr,num_lab_procedures,num_medications,time_in_hospital,number_diagnoses,num_procedures,insulin,gender_Male,A1Cresult,...,diag_3_Other,number_inpatient_squared,metformin,diag_1_Other,medical_specialty_Unknown,race_Caucasian,change_No,age_[60-70),number_inpatient,readmitted
0,2278392.0,8222157.0,-0.094292,-1.769617,-1.115670,-3.110153,-0.811261,0.0,False,0.0,...,False,-0.144824,0.0,True,False,True,True,False,-0.292997,0.0
1,149190.0,55629189.0,0.810496,0.281724,-0.433998,0.887364,-0.811261,2.0,False,0.0,...,True,-0.144824,0.0,True,True,True,False,False,-0.292997,1.0
2,64410.0,86047875.0,-1.602273,-0.321612,-0.774834,-0.611705,2.034513,0.0,False,0.0,...,True,0.223657,0.0,True,True,False,True,False,1.369158,0.0
3,500364.0,82442376.0,0.056506,0.040390,-0.774834,-0.112016,-0.242107,2.0,True,0.0,...,False,-0.144824,0.0,True,True,True,False,False,-0.292997,0.0
4,16680.0,42519267.0,0.408368,-0.924947,-1.115670,-1.111395,-0.811261,1.0,True,0.0,...,False,-0.144824,0.0,False,True,True,False,False,-0.292997,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
69970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
69978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
69983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


<h2 style="text-align: center; color: #2E8B57;"><b>Data Preparation and Feature Engineering Process</b></h2>

<p style="color: #006400; font-size: 1.1em;">This process outlines the steps undertaken for preprocessing, feature scaling, encoding, and selection to prepare the dataset for machine learning tasks.</p>

<hr style="border: 1px solid #2E8B57;">

<h3 style="color: #2E8B57;">1. 📚 Import Necessary Libraries</h3>
<ul style="color: #006400; font-size: 1.05em; list-style: none; padding-left: 0;">
    <li>🔹 Essential Python libraries such as <b>pandas</b>, <b>numpy</b>, and <b>sklearn</b> are imported for:</li>
    <ul style="color: #006400; font-size: 1em;">
        <li>Data preprocessing</li>
        <li>Scaling</li>
        <li>Feature selection</li>
    </ul>
</ul>

<h3 style="color: #2E8B57;">2. 🗃️ Load Preprocessed Data</h3>
<ul style="color: #006400; font-size: 1.05em; list-style: none; padding-left: 0;">
    <li>🔸 The preprocessed data is loaded from <b>preprocessed.pkl</b>, which contains a cleaned and structured dataset.</li>
    <li>🔸 The dataset is split into features (<b>X</b>) and the target variable (<b>y</b>).</li>
</ul>

<h3 style="color: #2E8B57;">3. 🔄 Encoding the Target Variable</h3>
<ul style="color: #006400; font-size: 1.05em; list-style: none; padding-left: 0;">
    <li>🔹 If the target variable (<b>readmitted</b>) contains non-numeric labels, they are encoded into numeric values using <b>LabelEncoder</b>.</li>
    <li>🔹 This step is essential as most machine learning algorithms require numeric inputs.</li>
</ul>

<h3 style="color: #2E8B57;">4. 📏 Feature Scaling</h3>
<ul style="color: #006400; font-size: 1.05em; list-style: none; padding-left: 0;">
    <li>🔸 <b>Numerical Columns Identification:</b> Identifies numerical columns in <b>X</b> using <code>select_dtypes(include=[np.number])</code>.</li>
    <li>🔸 <b>Standard Scaling:</b> Applies <b>StandardScaler</b> to ensure numerical data has a mean of 0 and standard deviation of 1.</li>
    <li>🔸 <b>Importance of Scaling:</b></li>
    <ul style="color: #006400; font-size: 1em;">
        <li>Ensures features are on the same scale.</li>
        <li>Prevents features with larger magnitudes from dominating the model's learning process.</li>
    </ul>
</ul>

<h3 style="color: #2E8B57;">5. 🔧 One-Hot Encoding for Categorical Features</h3>
<ul style="color: #006400; font-size: 1.05em; list-style: none; padding-left: 0;">
    <li>🔹 Converts categorical columns into numerical format using <code>pd.get_dummies</code>.</li>
    <li>🔹 Represents each category as a binary feature, ensuring compatibility with machine learning algorithms.</li>
</ul>

<h3 style="color: #2E8B57;">6. 🎯 Feature Selection</h3>
<ul style="color: #006400; font-size: 1.05em; list-style: none; padding-left: 0;">
    <li>🔸 <b>Highly Correlated Features:</b></li>
    <ul style="color: #006400; font-size: 1em;">
        <li>Calculates a correlation matrix for numerical columns.</li>
        <li>Removes features with a high correlation (>0.9) to reduce multicollinearity.</li>
    </ul>
    <li>🔸 <b>Feature Importance via Random Forest:</b> Retains the top <b>k</b> features (e.g., <b>k=20</b>) based on feature importance scores.</li>
    <li>🔸 <b>Chi-Square Test for Categorical Features:</b> Evaluates independence between categorical features and the target variable to select the most relevant ones.</li>
</ul>

<h3 style="color: #2E8B57;">7. 💾 Save Transformed Dataset</h3>
<ul style="color: #006400; font-size: 1.05em; list-style: none; padding-left: 0;">
    <li>🔹 The final transformed dataset (features + target) is saved to <b>transformed_data.pkl</b> for downstream tasks.</li>
</ul>

<p style="color: #006400; font-size: 1.1em;"><b>Conclusion:</b> These steps ensure that the dataset is ready for modeling by handling categorical variables, scaling numerical features, and selecting the most impactful features. This process establishes a robust foundation for building accurate predictive models.</p>


<h1 style="text-align: center; color: blue;"><b>Model Training and Evaluation</b></h1>
<h2 style="text-align: center; color: blue;"><b>Random Forest Classifier for Readmission Prediction</b></h2>

In [25]:
print(f"X.dtypes\n",X.dtypes)
print()
print(f"y.dtype\n",y.dtype)

X.dtypes
 encounter_id                   int64
patient_nbr                    int64
num_lab_procedures           float64
num_medications              float64
time_in_hospital             float64
number_diagnoses             float64
num_procedures               float64
insulin                        int64
gender_Male                     bool
A1Cresult                      int64
diag_2_Other                    bool
diag_3_Other                    bool
number_inpatient_squared     float64
metformin                      int64
diag_1_Other                    bool
medical_specialty_Unknown       bool
race_Caucasian                  bool
change_No                       bool
age_[60-70)                     bool
number_inpatient             float64
dtype: object

y.dtype
 int64


In [27]:
print(f"X.isnull()\n", X.isnull().sum())
print()
# Check for null values in y (NumPy array)
print(f"y.isnull()\n", np.isnan(y).sum())

X.isnull()
 encounter_id                 0
patient_nbr                  0
num_lab_procedures           0
num_medications              0
time_in_hospital             0
number_diagnoses             0
num_procedures               0
insulin                      0
gender_Male                  0
A1Cresult                    0
diag_2_Other                 0
diag_3_Other                 0
number_inpatient_squared     0
metformin                    0
diag_1_Other                 0
medical_specialty_Unknown    0
race_Caucasian               0
change_No                    0
age_[60-70)                  0
number_inpatient             0
dtype: int64

y.isnull()
 0


In [34]:
# Separate features and target variable
X = transformed_data_df.drop(columns=['readmitted'])
y = transformed_data_df['readmitted']

# Split data into training and testing sets
#from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print a message indicating the purpose of the testing set
print("Testing set: Used for evaluating the model's performance")

Testing set: Used for evaluating the model's performance


In [35]:
# Check for NaN values in y
print("Missing values in y:", y.isnull().sum())

# Drop rows with NaN in y
valid_indices = y.dropna().index
X = X.loc[valid_indices]
y = y.loc[valid_indices]

# Now split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Missing values in y: 19332


<h1 style="text-align: center; color: blue;"><b>Model Training and Evaluation</b></h1>
<h2 style="text-align: center; color: blue;"><b>Plain Decision/Binary Tree  for Readmission Prediction</b></h2>

In [40]:
# Separate features and target variable
X = transformed_data_df.drop(columns=['readmitted'])  # assuming 'readmitted' is the target variable
y = transformed_data_df['readmitted']

# Check for NaN values in y and remove them
print("Missing values in y:", y.isnull().sum())
valid_indices = y.dropna().index
X = X.loc[valid_indices]
y = y.loc[valid_indices]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train a Decision Tree model
decision_tree = DecisionTreeClassifier(random_state=42, max_depth=5)  # Adjust max_depth to control overfitting
decision_tree.fit(X_train, y_train)

# Make predictions
y_pred = decision_tree.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Missing values in y: 19332
Accuracy: 0.5923227127684907

Classification Report:
               precision    recall  f1-score   support

         0.0       0.61      0.89      0.72     12455
         1.0       0.50      0.17      0.25      8542

    accuracy                           0.59     20997
   macro avg       0.55      0.53      0.48     20997
weighted avg       0.56      0.59      0.53     20997


Confusion Matrix:
 [[11023  1432]
 [ 7128  1414]]


In [43]:
# Select only relevant numeric features
numeric_features = [
    'num_lab_procedures', 'num_medications', 'time_in_hospital',
    'number_diagnoses', 'num_procedures', 'number_inpatient'
]
X = transformed_data_df[numeric_features]
y = transformed_data_df['readmitted']

# Handle NaN values in X and y together
X = X.fillna(X.median())
valid_indices = y.dropna().index  # Drop NaN values from y and align with X
X = X.loc[valid_indices]
y = y.loc[valid_indices]

# Ensure X and y have the same number of samples
assert len(X) == len(y), "X and y have inconsistent lengths!"

# Balance classes using SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Perform hyperparameter tuning and training
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42, class_weight='balanced'), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
Accuracy: 0.558948806557904

Classification Report:
               precision    recall  f1-score   support

         0.0       0.54      0.75      0.63     12468
         1.0       0.59      0.37      0.46     12418

    accuracy                           0.56     24886
   macro avg       0.57      0.56      0.54     24886
weighted avg       0.57      0.56      0.54     24886


Confusion Matrix:
 [[9291 3177]
 [7799 4619]]


In [44]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Separate features and target variable
X = transformed_data_df.drop(columns=['readmitted'])  # assuming 'readmitted' is the target variable
y = transformed_data_df['readmitted']

# Check for NaN values in y and remove them
print("Missing values in y:", y.isnull().sum())
valid_indices = y.dropna().index
X = X.loc[valid_indices]
y = y.loc[valid_indices]

# Handle missing values in X if any
print("Missing values in X before imputation:", X.isnull().sum().sum())
X.fillna(X.median(), inplace=True)  # Replace NaN with median for numerical columns
print("Missing values in X after imputation:", X.isnull().sum().sum())

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Hyperparameter Tuning with Grid Search
param_grid = {
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'criterion': ['gini', 'entropy']
}
decision_tree = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(decision_tree, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, y_train)

# Best parameters and model
print("Best Parameters:", grid_search.best_params_)
best_decision_tree = grid_search.best_estimator_

# Train the best model
best_decision_tree.fit(X_train_scaled, y_train)

# Predictions
y_pred = best_decision_tree.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("\nDecision Tree Model (After Improvements)")
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Feature Importance Analysis
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_decision_tree.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nTop Features Contributing to the Model:\n", feature_importances.head(10))

Missing values in y: 19332
Missing values in X before imputation: 386640
Missing values in X after imputation: 0
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5}

Decision Tree Model (After Improvements)
Accuracy: 0.5883629349835249

Classification Report:
               precision    recall  f1-score   support

         0.0       0.56      0.86      0.68     12468
         1.0       0.70      0.31      0.43     12418

    accuracy                           0.59     24886
   macro avg       0.63      0.59      0.55     24886
weighted avg       0.63      0.59      0.55     24886


Confusion Matrix:
 [[10770  1698]
 [ 8546  3872]]

Top Features Contributing to the Model:
                Feature  Importance
10        diag_2_Other    0.306597
8          gender_Male    0.223302
0         encounter_id    0.155998
11        diag_3_Other    0.059870
7              insulin    0.035

<h1 style="text-align: center; color: #1E90FF;"><b>Insights on Decision Tree Results</b></h1>
<hr style="border: 2px solid #4682B4;">

<h2 style="color: #4682B4;">📊 <b>Model Performance (Decision Tree After Improvements)</b></h2>
<ul style="color: #4B0082; font-size: 1.1em;">
    <li><b>Accuracy:</b> 0.59 - The model correctly classifies 59% of cases, showing moderate improvement.</li>
    <li><b>Precision:</b>
        <ul>
            <li><b>Class 0 (Not Readmitted):</b> 0.56 - 56% of predicted non-readmissions are correct.</li>
            <li><b>Class 1 (Readmitted):</b> 0.70 - 70% of predicted readmissions are correct, a significant improvement.</li>
        </ul>
    </li>
    <li><b>Recall:</b>
        <ul>
            <li><b>Class 0 (Not Readmitted):</b> 0.86 - High recall indicates the model effectively identifies non-readmitted patients.</li>
            <li><b>Class 1 (Readmitted):</b> 0.31 - Low recall means many readmitted patients are missed.</li>
        </ul>
    </li>
    <li><b>F1-Score:</b>
        <ul>
            <li><b>Class 0:</b> 0.68 - Balanced performance for the majority class.</li>
            <li><b>Class 1:</b> 0.43 - Shows room for improvement in handling the minority class.</li>
        </ul>
    </li>
</ul>

<hr style="border: 1px dashed #4682B4;">

<h2 style="color: #4682B4;">📋 <b>Confusion Matrix</b></h2>
<ul style="color: #4B0082; font-size: 1.1em;">
    <li><b>True Positives (TP):</b> 3872 - Correctly predicted readmitted cases.</li>
    <li><b>False Negatives (FN):</b> 8546 - Readmitted cases missed by the model.</li>
    <li><b>True Negatives (TN):</b> 10770 - Correctly predicted non-readmitted cases.</li>
    <li><b>False Positives (FP):</b> 1698 - Non-readmitted cases predicted as readmitted.</li>
</ul>
<p style="color: #4B0082; font-size: 1.1em;">The high number of <b>False Negatives (8,546)</b> highlights the need to improve recall for readmitted patients.</p>

<hr style="border: 1px dashed #4682B4;">

<h2 style="color: #4682B4;">📈 <b>Feature Importance Analysis</b></h2>
<ul style="color: #4B0082; font-size: 1.1em;">
    <li><b>Top Features:</b></li>
    <ul>
        <li><b>diag_2_Other:</b> 0.3066 - Most significant feature, reflecting the role of diagnosis codes.</li>
        <li><b>gender_Male:</b> 0.2233 - Gender plays a key role, particularly male patients.</li>
        <li><b>encounter_id:</b> 0.1560 - Requires investigation to ensure no data leakage.</li>
        <li><b>diag_3_Other:</b> 0.0599 - Another diagnosis-related feature.</li>
        <li><b>insulin:</b> 0.0359 - Reflects the role of diabetes management.</li>
        <li><b>num_lab_procedures:</b> 0.0309 - Key indicator of health evaluations.</li>
        <li><b>number_diagnoses:</b> 0.0253 - Patients with multiple diagnoses are at higher risk of readmission.</li>
    </ul>
</ul>

<hr style="border: 1px dashed #4682B4;">

<h2 style="color: #4682B4;">🔍 <b>Key Observations</b></h2>
<ul style="color: #4B0082; font-size: 1.1em;">
    <li>📉 <b>Class Imbalance:</b> Precision for readmissions improved, but recall remains low, highlighting the imbalance challenge.</li>
    <li>🔑 <b>Feature Contributions:</b> Diagnosis codes and gender are the most influential, with patient and encounter attributes requiring validation for potential bias.</li>
    <li>🚨 <b>Room for Improvement:</b> Recall for readmitted patients needs further enhancement.</li>
</ul>

<hr style="border: 1px dashed #4682B4;">

<h2 style="color: #4682B4;">💡 <b>Recommendations</b></h2>
<ul style="color: #4B0082; font-size: 1.1em;">
    <li>📊 <b>Improve Recall:</b>
        <ul>
            <li>Implement ensemble models like <b>Random Forest</b> or <b>Gradient Boosting</b>.</li>
            <li>Use techniques like <b>Cost-Sensitive Learning</b> or advanced resampling (e.g., <b>ADASYN</b>).</li>
        </ul>
    </li>
    <li>🔧 <b>Refine Features:</b> Engineer new features (e.g., interaction terms, aggregate metrics).</li>
    <li>🔄 <b>Tune Hyperparameters:</b> Optimize tree depth, splits, and explore broader hyperparameter ranges.</li>
    <li>🚀 <b>Evaluate Alternate Metrics:</b> Use <b>ROC-AUC</b> to better balance precision and recall trade-offs.</li>
</ul>
<p style="color: #4B0082; font-size: 1.1em;"><b>Conclusion:</b> While the Decision Tree shows moderate improvement, addressing class imbalance and leveraging advanced models can significantly enhance its predictive power for readmitted patients.</p>
